## Registration Service demo
compatible with EDC v0.10.1

In [ ]:
import requests
import json

from dataspace_apis import *

### Demo setup

In [ ]:

IS_LOCALHOST_DEPLOYMENT = True

# for paas:
PROVIDER_URL = "https://provider-edc-connector.apps.paas-dev.psnc.pl"
CONSUMER_URL = "https://consumer-edc-connector.apps.paas-dev.psnc.pl"
REGISTRATION_SERVICE_URL = "https://registration-service-backend-edc-connector.apps.paas-dev.psnc.pl"
FEDERATED_CATALOG_BASE_URL = "https://federatedcatalog-edc-connector.apps.paas-dev.psnc.pl"
CONSUMER_BACKEND_URL = "https://consback-edc-connector.apps.paas-dev.psnc.pl"

## for local:
LOCALHOST = "http://localhost"
CONSUMER_CONTAINER = "http://consumer-connector"
PROVIDER_CONTAINER = "http://provider-connector"
REGISTRATION_SERVICE_CONTAINER = "http://registration-service-backend"
FEDERATED_CATALOG_BASE_CONTAINER = "http://federated-catalog"
CONSUMER_BACKEND_CONTAINER = "http://consumer-backend"

if (IS_LOCALHOST_DEPLOYMENT):
    CONSUMER_URL = LOCALHOST
    PROVIDER_URL = LOCALHOST
    REGISTRATION_SERVICE_URL = LOCALHOST
    FEDERATED_CATALOG_BASE_URL = LOCALHOST

In [ ]:
PROVIDER_API = f"{PROVIDER_URL}/api"
PROVIDER_CONTROL = f"{PROVIDER_URL}/control"
PROVIDER_MANAGEMENT = f"{PROVIDER_URL}/management"
PROVIDER_PROTOCOL = f"{PROVIDER_URL}/protocol"
PROVIDER_PUBLIC = f"{PROVIDER_URL}/public"

CONSUMER_API = f"{CONSUMER_URL}/api"
CONSUMER_CONTROL = f"{CONSUMER_URL}/control"
CONSUMER_MANAGEMENT = f"{CONSUMER_URL}/management"
CONSUMER_PROTOCOL = f"{CONSUMER_URL}/protocol"
CONSUMER_PUBLIC = f"{CONSUMER_URL}/public"

AUTHORITY_URL = f"{REGISTRATION_SERVICE_URL}/authority"
PARTICIPANTS_ENDPOINT_URL = f"{AUTHORITY_URL}/registry/participants"

CONSUMER_BACKEND_EDR = f"{CONSUMER_BACKEND_URL}/edr-endpoint"
FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}/catalog"

if (IS_LOCALHOST_DEPLOYMENT):
    PROVIDER_API = f"{PROVIDER_URL}:19191/api"
    PROVIDER_CONTROL = f"{PROVIDER_URL}:19192/control"
    PROVIDER_MANAGEMENT = f"{PROVIDER_URL}:19193/management"
    PROVIDER_PROTOCOL = f"{PROVIDER_URL}:19194/protocol"
    PROVIDER_PUBLIC = f"{PROVIDER_URL}:19291/public"

    CONSUMER_API = f"{CONSUMER_URL}:29191/api"
    CONSUMER_CONTROL = f"{CONSUMER_URL}:29192/control"
    CONSUMER_MANAGEMENT = f"{CONSUMER_URL}:29193/management"
    CONSUMER_PROTOCOL = f"{CONSUMER_URL}:29194/protocol"
    CONSUMER_PUBLIC = f"{CONSUMER_URL}:29291/public"

    AUTHORITY_URL = f"{REGISTRATION_SERVICE_URL}:38182/authority"
    PARTICIPANTS_ENDPOINT_URL = f"{AUTHORITY_URL}/registry/participants"

    CONSUMER_BACKEND_EDR = f"{CONSUMER_BACKEND_CONTAINER}:4000/edr-endpoint"
    FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}:9181/catalog"

In [ ]:
provider_control_internal = PROVIDER_CONTROL.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_public_internal = PROVIDER_PUBLIC.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_protocol_internal = f"{PROVIDER_PROTOCOL}".replace(LOCALHOST, PROVIDER_CONTAINER)

default_headers = {
    "Content-Type": "application/json",
    "x-api-key": "edc",
}

# participant for tests (add/update/delete)
did = "consumer"
protocolUrl = CONSUMER_PROTOCOL

## Registration Service main operations

### Show dataspace participants

In [ ]:
def show_participants():
    return requests.get(PARTICIPANTS_ENDPOINT_URL, headers=default_headers).json()

participants = show_participants()
print("Stored participants in the Registration Service database:")
print(participants)

### Add a new participant

In [ ]:
def add_participant(did, protocolUrl):
    requests.post(f"{PARTICIPANTS_ENDPOINT_URL}?did={did}&protocolUrl={protocolUrl}", headers=default_headers)

add_participant(did, protocolUrl)

In [ ]:
show_participants()

In [ ]:
fetch_catalog(FEDERATED_CATALOG_URL, default_headers).json()

### Update participant's status

In [ ]:
"""
Available statuses:
---
ONBOARDING_INITIATED(0), // onboarding request received
AUTHORIZING(100), // verifying participants credentials
AUTHORIZED(200), // participant is authorized
ONBOARDED(300), // participant is fully onboarded
DENIED(400), // participant onboarding request denied
FAILED(-1), // participant onboarding failed
DELETED(-100);
"""

def update_participant_status(did, newStatus):
    requests.patch(f"{PARTICIPANTS_ENDPOINT_URL}/{did}?status={newStatus}", headers=default_headers)

update_participant_status(did, "DENIED")

In [ ]:
update_participant_status(did, "ONBOARDED")

In [ ]:
show_participants()

### Update participant's claims

In [ ]:
def update_claims(did, claims):
    requests.patch(
        headers=default_headers,
        data=json.dumps(claims),
        url=f"{PARTICIPANTS_ENDPOINT_URL}/{did}/claims")

update_claims(did, { "region": "pl" })

In [ ]:
update_claims(did, { "region": "pl" })

In [ ]:
show_participants()

## Perform asset transfer

### Create example asset, policy and policy definition

In [ ]:
asset_id = "registration-service-notebook-example-asset"

asset = create_asset(asset_id, PROVIDER_MANAGEMENT, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
asset.status_code

In [ ]:
policy_id = "registration-service-notebook-example-policy"
allowed_policy_region = "pl"

allowed_region_rule = {
    "action": "use", 
    "constraint": { 
        "@type": "AtomicConstraint", 
        "leftOperand": "https://w3id.org/edc/v0.0.1/ns/regionLocation", 
        "operator": "odrl:eq", 
        "rightOperand": allowed_policy_region 
    }
}

policy = create_policy(policy_id, PROVIDER_MANAGEMENT, default_headers, permissions=[allowed_region_rule])

# expected 200 or 409 (already exists) because the @id is fixed in the example
print(policy.status_code)

In [ ]:
contract_definition_id = "registration-service-notebook-example-contract-definition"

contract_definition = create_contract_definition(contract_definition_id, PROVIDER_MANAGEMENT, asset_id, policy_id, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
contract_definition.status_code

### Fetch catalog, negotiate and perform transfer

In [ ]:
import time

# 1. fetch catalog
fetched_catalog = fetch_catalog(FEDERATED_CATALOG_URL, default_headers)
print(f"1. Catalog fetch: {fetched_catalog.status_code}")

# 2. check if there is an existing negotiation
existing_negotiation = check_existing_negotiation(asset_id, CONSUMER_MANAGEMENT, default_headers)
contract_agreement_id = ""

if existing_negotiation is not None:
    # 2A. Use existing negotiation
    contract_agreement_id = existing_negotiation
    print(f"Negotiation for asset \"{asset_id}\" already exists")
else:
    offer_id = get_offer_id(fetched_catalog.json(), asset_id)
    print("OfferId:", offer_id)

    # 2B. Negotitate a contract
    negotiated_contract = negotiate_contract(
        offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [allowed_region_rule], default_headers)
    # If there's policy with rules then negotiated_contract = negotiate_contract(offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [allowed_region_rule], default_headers)
    print(f"Negotiate a contract: {negotiated_contract.status_code}")

    # wait a minute until the negotiation will be finalized (automatic interval process takes maximum 60 secondes to load new contracts)
    print("... Waiting for negotiation to save ...")
    time.sleep(60)

    contract_negotiation_id = negotiated_contract.json()["@id"]
    contract_agreement = get_contract_agreement_id(contract_negotiation_id, CONSUMER_MANAGEMENT, default_headers)
    print(contract_agreement.json())

    if ("contractAgreementId" in contract_agreement.json()):
        contract_agreement_id = contract_agreement.json()["contractAgreementId"]

print("[@AgreementId]:", contract_agreement_id)

# Check for existing transfer for given asset and agreement
existing_transfer = check_existing_transfer(asset_id, contract_agreement_id, CONSUMER_BACKEND_EDR, CONSUMER_MANAGEMENT, default_headers)

# 3. transfer asset
if existing_transfer is not None:
    print(f"Requested transfer already exists: {existing_transfer}")
    requested_transfer = get_transfer(existing_transfer, CONSUMER_MANAGEMENT, default_headers)
else:
    requested_transfer = request_consumer_pull_transfer(
        "provider",
        CONSUMER_MANAGEMENT,
        CONSUMER_BACKEND_EDR,
        provider_protocol_internal,
        contract_agreement_id,
        default_headers
    )

print(f"3. Transfer an asset: {requested_transfer.status_code}")
print(requested_transfer.json())

In [ ]:
transfer_id = requested_transfer.json()["@id"]

get_transfer_state(CONSUMER_MANAGEMENT, transfer_id, default_headers)

### Remove a participant from the dataspace

In [ ]:
def remove_participant(did):
    requests.delete(f"{PARTICIPANTS_ENDPOINT_URL}/{did}")

remove_participant(did)

In [ ]:
show_participants()

In [ ]:
fetch_catalog(FEDERATED_CATALOG_URL, default_headers).json()